# DISCRETE RHO:

# FOR EACH BIOREPLICATE:

# MLE ESTIMATION OF P = P(MUTATION)

In [1]:
import numpy
import scipy.misc as sm
import math

# for small numbers

In [2]:
def f(rho,n,R,q):
    #known: n = reads w/ obs indels, R = total reads, q = fraction neg control reads w/ indels
    #maximize f over rho to get MLE of p = P(true indel) = rho/R (according to Binomial Error Model)
    #rho = reads w/ true indels element of {0,1,...,n} <= R
    #R is natural number
    #q in [0,1]
    return sm.factorial(R-rho)/sm.factorial(n-rho)*q**(-rho)

In [3]:
#works
print(f(1,4,5,.5))
print(8)
print(f(2,4,5,.5))
print(12)

8.0
8
12.0
12


# for large numbers

In [4]:
def approxlnf(rho,n,R,q):
    #returns ln(f), using Stirling's approx for large factorials
    #typical Hsu data:
    #R~9000 large compared to n~100, rho~90
    #q~.01 small
    
    return .5*(numpy.log(2*numpy.pi))+(R-rho+.5)*numpy.log(R-rho)-numpy.log(sm.factorial(n-rho))-(R-rho)-rho*numpy.log(q)

In [5]:
print(approxlnf(90,100,9000,.01))
print(numpy.log(2*numpy.pi)/2+8910.5*numpy.log(8910)-numpy.log(sm.factorial(10))-8910-90*numpy.log(.01)) #works
print(approxlnf(100,100,9000,.01))
print(numpy.log(2*numpy.pi)/2+8900.5*numpy.log(8900)-8900-100*numpy.log(.01)) #0! works
print(numpy.log(f(90,100,200,.6)))
print(approxlnf(90,100,200,.6)) #approx works
print(numpy.log(f(92,100,200,.6)))
print(approxlnf(92,100,200,.6)) #trend works

72530.6493348
72530.6493348
72500.8612063
72500.8612063
441.192670093
441.191912519
437.322302763
437.32153116


# derivation see MLE.pdf

In [5]:
def pMLE(n,R,q):
    #x element of {0,1,...,n-1}
    #f(x+1)/f(x) strictly mono dec w/x --> f(rho) concave down
    #return MLE of p = rho/R that maximizes f(rho,n,R,q)
    
    #reference
    #rho=numpy.arange(0,n+1,1)
    #p=rho/R
    #y=approxlnf(rho,n,R,q)
    #rhopyhoriz=numpy.array([rho,p,y])
    #rhopyvert=numpy.transpose(rhopyhoriz)
    #print("[rho, p, ~ln(f)] = ")
    #print(rhopyvert)
    #print("pMLE = ")
    
    #result
    x=numpy.arange(0,n,1)
    #A: f(x+1)/f(x) < 1 always
    if q>n/R:
        return numpy.array([0])
    else:
        x0=(n-R*q)/(1-q)
        #D: f(x+1)/f(x) > 1 always
        if x0 > n-1:
            return numpy.array([n/R])
        else:
            x1=math.floor(x0)
            x2=math.ceil(x0)
            #C: f(x+1)/f(x) = 1 at some x
            if x1==x0:
                rhomax=numpy.array([x0,x0+1])
                return rhomax/R
            #D: f(x+1)/f(x) < 1 at some x, never = 1
            else:
                if approxlnf(x2,n,R,q)>approxlnf(x1,n,R,q):
                    return numpy.array([x2/R])
                elif approxlnf(x2,n,R,q)==approxlnf(x1,n,R,q):
                    return numpy.array([x1,x2])/R
                else:
                    return numpy.array([x1/R])

# example A

In [7]:
pMLE(100,9000,.02) #works

[rho, p, ~ln(f)] = 
[[  0.00000000e+00   0.00000000e+00   7.25865508e+04]
 [  1.00000000e+00   1.11111111e-04   7.25859630e+04]
 [  2.00000000e+00   2.22222222e-04   7.25853652e+04]
 [  3.00000000e+00   3.33333333e-04   7.25847575e+04]
 [  4.00000000e+00   4.44444444e-04   7.25841396e+04]
 [  5.00000000e+00   5.55555556e-04   7.25835114e+04]
 [  6.00000000e+00   6.66666667e-04   7.25828729e+04]
 [  7.00000000e+00   7.77777778e-04   7.25822239e+04]
 [  8.00000000e+00   8.88888889e-04   7.25815643e+04]
 [  9.00000000e+00   1.00000000e-03   7.25808940e+04]
 [  1.00000000e+01   1.11111111e-03   7.25802129e+04]
 [  1.10000000e+01   1.22222222e-03   7.25795209e+04]
 [  1.20000000e+01   1.33333333e-03   7.25788178e+04]
 [  1.30000000e+01   1.44444444e-03   7.25781035e+04]
 [  1.40000000e+01   1.55555556e-03   7.25773779e+04]
 [  1.50000000e+01   1.66666667e-03   7.25766409e+04]
 [  1.60000000e+01   1.77777778e-03   7.25758922e+04]
 [  1.70000000e+01   1.88888889e-03   7.25751319e+04]
 [  1.80

array([0])

# example B

In [8]:
pMLE(100,9000,.01) #works

[rho, p, ~ln(f)] = 
[[  0.00000000e+00   0.00000000e+00   7.25865508e+04]
 [  1.00000000e+00   1.11111111e-04   7.25866561e+04]
 [  2.00000000e+00   2.22222222e-04   7.25867515e+04]
 [  3.00000000e+00   3.33333333e-04   7.25868369e+04]
 [  4.00000000e+00   4.44444444e-04   7.25869122e+04]
 [  5.00000000e+00   5.55555556e-04   7.25869771e+04]
 [  6.00000000e+00   6.66666667e-04   7.25870318e+04]
 [  7.00000000e+00   7.77777778e-04   7.25870759e+04]
 [  8.00000000e+00   8.88888889e-04   7.25871095e+04]
 [  9.00000000e+00   1.00000000e-03   7.25871324e+04]
 [  1.00000000e+01   1.11111111e-03   7.25871444e+04]
 [  1.10000000e+01   1.22222222e-03   7.25871455e+04]
 [  1.20000000e+01   1.33333333e-03   7.25871356e+04]
 [  1.30000000e+01   1.44444444e-03   7.25871144e+04]
 [  1.40000000e+01   1.55555556e-03   7.25870820e+04]
 [  1.50000000e+01   1.66666667e-03   7.25870381e+04]
 [  1.60000000e+01   1.77777778e-03   7.25869826e+04]
 [  1.70000000e+01   1.88888889e-03   7.25869154e+04]
 [  1.80

array([ 0.00122222])

# INVERSE PROBLEM: ESTIMATION OF N

In [87]:
def nINV(p,R,q):
    #known: p = P(true indel), R = total reads, q = fraction neg control reads w/ indels
    #find: n s.t. pMLE = p estimate is closest to p
    #return: n, errori = |pi_n - pi|, m_n = number of p_n estimates, pi_n (i = 1,2)
    
    rho=R*p
    #return all paired p estimates flanking p, single p estimates closest to p
    nval=numpy.array([])
    error1=numpy.array([])
    error2=numpy.array([])
    mval=numpy.array([])
    p1=numpy.array([])
    p2=numpy.array([])
    
    #all single p estimates
    n=numpy.array([[]])
    errorn=numpy.array([])
    pn=numpy.array([])
    
    k=math.floor(rho)
    while k<R+1:
        pmle=pMLE(k,R,q)
        if pmle.size==1:
            pmle=pmle[0]
            pn=numpy.append(pn,pmle)
            error=numpy.abs(pmle-p)
            errorn=numpy.append(errorn,error)
            n=numpy.append(n,k)
        else:
            if (p >= pmle[0]) & (p <= pmle[1]): #accept any paired p estimates flanking p
                p1=numpy.append(p1,pmle[0])
                p2=numpy.append(p2,pmle[1])
                mval=numpy.append(mval,2)
                error=numpy.abs(pmle-p)
                error1=numpy.append(error1,error[0])
                error2=numpy.append(error2,error[1])
                nval=numpy.append(nval,k)
        k+=1
    
    #single p estimates closest to p
    i=numpy.argmin(errorn) #index of first min
    nval=numpy.append(nval,n[i])
    error1=numpy.append(error1,errorn[i])
    error2=numpy.append(error2,errorn[i])
    mval=numpy.append(mval,1)
    p1=numpy.append(p1,pn[i])
    p2=numpy.append(p2,pn[i])
    k=i+1 #find other equal mins
    while k<n.size:
        if errorn[k]==errorn[i]:
            nval=numpy.append(nval,n[k])
            error1=numpy.append(error1,errorn[k])
            error2=numpy.append(error2,errorn[k])
            mval=numpy.append(mval,1)
            p1=numpy.append(p1,pn[k])
            p2=numpy.append(p2,pn[k])
        k+=1
            
    #nemphoriz=numpy.array([nval,error1,error2,mval,p1,p2])
    #nemp=nemphoriz.transpose()
    #print("[n, error1, error2, number of pMLE, pMLE1, pMLE2] = ")
    #print(nemp)
    
    y=numpy.array([nval,error1,error2])
    #print("[n, error1, error2] =")
    #print(y)
    return(y)

# example A

In [86]:
nINV(0,9000,.02)
print("n = 100") #works

[n, error1, error2] =
[[  1.80000000e+02   0.00000000e+00   1.00000000e+00   2.00000000e+00
    3.00000000e+00   4.00000000e+00   5.00000000e+00   6.00000000e+00
    7.00000000e+00   8.00000000e+00   9.00000000e+00   1.00000000e+01
    1.10000000e+01   1.20000000e+01   1.30000000e+01   1.40000000e+01
    1.50000000e+01   1.60000000e+01   1.70000000e+01   1.80000000e+01
    1.90000000e+01   2.00000000e+01   2.10000000e+01   2.20000000e+01
    2.30000000e+01   2.40000000e+01   2.50000000e+01   2.60000000e+01
    2.70000000e+01   2.80000000e+01   2.90000000e+01   3.00000000e+01
    3.10000000e+01   3.20000000e+01   3.30000000e+01   3.40000000e+01
    3.50000000e+01   3.60000000e+01   3.70000000e+01   3.80000000e+01
    3.90000000e+01   4.00000000e+01   4.10000000e+01   4.20000000e+01
    4.30000000e+01   4.40000000e+01   4.50000000e+01   4.60000000e+01
    4.70000000e+01   4.80000000e+01   4.90000000e+01   5.00000000e+01
    5.10000000e+01   5.20000000e+01   5.30000000e+01   5.40000000e+0

In [35]:
print(pMLE(181,9000,.02)) #works
print("pMLE != 0")

[ 0.00011111  0.00022222]
pMLE != 0


In [36]:
print(pMLE(180,9000,.02)) #works
print("pMLE = 0")

[ 0.          0.00011111]
pMLE = 0


# example B

In [69]:
nINV(0.00122222,9000,.01)
print("n = 100") #works

[n, error1, error2] =
[[  1.00000000e+02]
 [  2.22222222e-09]
 [  2.22222222e-09]]
n = 100


# HSU DATA

In [7]:
from openpyxl import Workbook
from openpyxl import load_workbook

In [90]:
def ndata(wbname):
    #given excel file wbname (data for a target)
    #compute n1, error1-1, error1-2, n2, error2-1, error2-2 (n for both bioreplicates, error for both p estimates for both bioreplicates)
    #write into excel file
    
    wb=load_workbook(wbname)
    ws=wb.active
    ws['AC1'].value="n1"
    ws['AD1'].value="error1-1"
    ws['AE1'].value="error1-2"
    ws['AF1'].value="n2"
    ws['AG1'].value="error2-1"
    ws['AH1'].value="error2-2"
    
    
    q1=ws.cell(row=59,column=8).value
    q2=ws.cell(row=59,column=20).value
    k=2
    while k<59:
        p=ws.cell(row=k,column=28).value
        R1=ws.cell(row=k,column=9).value
        R2=ws.cell(row=k,column=21).value
        
        ws.cell(row=k,column=29).value=numpy.array_str(nINV(p,R1,q1)[:][0])
        ws.cell(row=k,column=30).value=numpy.array_str(nINV(p,R1,q1)[:][1])
        ws.cell(row=k,column=31).value=numpy.array_str(nINV(p,R1,q1)[:][2])
        print("k = ",k," rep = ",1)
        ws.cell(row=k,column=32).value=numpy.array_str(nINV(p,R2,q2)[:][0])
        ws.cell(row=k,column=33).value=numpy.array_str(nINV(p,R2,q2)[:][1])
        ws.cell(row=k,column=34).value=numpy.array_str(nINV(p,R2,q2)[:][2])
        print("k = ",k," rep = ",2)
        
        k+=1
    
    wb.save(wbname)

In [91]:
ndata('supp5emx1.1.xlsx') #takes 3 minutes

k =  2  rep =  1
k =  2  rep =  2
k =  3  rep =  1
k =  3  rep =  2
k =  4  rep =  1
k =  4  rep =  2
k =  5  rep =  1
k =  5  rep =  2
k =  6  rep =  1
k =  6  rep =  2
k =  7  rep =  1
k =  7  rep =  2
k =  8  rep =  1
k =  8  rep =  2
k =  9  rep =  1
k =  9  rep =  2
k =  10  rep =  1
k =  10  rep =  2
k =  11  rep =  1
k =  11  rep =  2
k =  12  rep =  1
k =  12  rep =  2
k =  13  rep =  1
k =  13  rep =  2
k =  14  rep =  1
k =  14  rep =  2
k =  15  rep =  1
k =  15  rep =  2
k =  16  rep =  1
k =  16  rep =  2
k =  17  rep =  1
k =  17  rep =  2
k =  18  rep =  1
k =  18  rep =  2
k =  19  rep =  1
k =  19  rep =  2
k =  20  rep =  1
k =  20  rep =  2
k =  21  rep =  1
k =  21  rep =  2
k =  22  rep =  1
k =  22  rep =  2
k =  23  rep =  1
k =  23  rep =  2
k =  24  rep =  1
k =  24  rep =  2
k =  25  rep =  1
k =  25  rep =  2
k =  26  rep =  1
k =  26  rep =  2
k =  27  rep =  1
k =  27  rep =  2
k =  28  rep =  1
k =  28  rep =  2
k =  29  rep =  1
k =  29  rep =  2
k =  30 

In [92]:
ndata('supp5emx1.2.xlsx')

k =  2  rep =  1
k =  2  rep =  2
k =  3  rep =  1
k =  3  rep =  2
k =  4  rep =  1
k =  4  rep =  2
k =  5  rep =  1
k =  5  rep =  2
k =  6  rep =  1
k =  6  rep =  2
k =  7  rep =  1
k =  7  rep =  2
k =  8  rep =  1
k =  8  rep =  2
k =  9  rep =  1
k =  9  rep =  2
k =  10  rep =  1
k =  10  rep =  2
k =  11  rep =  1
k =  11  rep =  2
k =  12  rep =  1
k =  12  rep =  2
k =  13  rep =  1
k =  13  rep =  2
k =  14  rep =  1
k =  14  rep =  2
k =  15  rep =  1
k =  15  rep =  2
k =  16  rep =  1
k =  16  rep =  2
k =  17  rep =  1
k =  17  rep =  2
k =  18  rep =  1
k =  18  rep =  2
k =  19  rep =  1
k =  19  rep =  2
k =  20  rep =  1
k =  20  rep =  2
k =  21  rep =  1
k =  21  rep =  2
k =  22  rep =  1
k =  22  rep =  2
k =  23  rep =  1
k =  23  rep =  2
k =  24  rep =  1
k =  24  rep =  2
k =  25  rep =  1
k =  25  rep =  2
k =  26  rep =  1
k =  26  rep =  2
k =  27  rep =  1
k =  27  rep =  2
k =  28  rep =  1
k =  28  rep =  2
k =  29  rep =  1
k =  29  rep =  2
k =  30 

In [93]:
ndata('supp5emx1.3.xlsx')

k =  2  rep =  1
k =  2  rep =  2
k =  3  rep =  1
k =  3  rep =  2
k =  4  rep =  1
k =  4  rep =  2
k =  5  rep =  1
k =  5  rep =  2
k =  6  rep =  1
k =  6  rep =  2
k =  7  rep =  1
k =  7  rep =  2
k =  8  rep =  1
k =  8  rep =  2
k =  9  rep =  1
k =  9  rep =  2
k =  10  rep =  1
k =  10  rep =  2
k =  11  rep =  1
k =  11  rep =  2
k =  12  rep =  1
k =  12  rep =  2
k =  13  rep =  1
k =  13  rep =  2
k =  14  rep =  1
k =  14  rep =  2
k =  15  rep =  1
k =  15  rep =  2
k =  16  rep =  1
k =  16  rep =  2
k =  17  rep =  1
k =  17  rep =  2
k =  18  rep =  1
k =  18  rep =  2
k =  19  rep =  1
k =  19  rep =  2
k =  20  rep =  1
k =  20  rep =  2
k =  21  rep =  1
k =  21  rep =  2
k =  22  rep =  1
k =  22  rep =  2
k =  23  rep =  1
k =  23  rep =  2
k =  24  rep =  1
k =  24  rep =  2
k =  25  rep =  1
k =  25  rep =  2
k =  26  rep =  1
k =  26  rep =  2
k =  27  rep =  1
k =  27  rep =  2
k =  28  rep =  1
k =  28  rep =  2
k =  29  rep =  1
k =  29  rep =  2
k =  30 

In [94]:
ndata('supp5emx1.6.xlsx')

k =  2  rep =  1
k =  2  rep =  2
k =  3  rep =  1
k =  3  rep =  2
k =  4  rep =  1
k =  4  rep =  2
k =  5  rep =  1
k =  5  rep =  2
k =  6  rep =  1
k =  6  rep =  2
k =  7  rep =  1
k =  7  rep =  2
k =  8  rep =  1
k =  8  rep =  2
k =  9  rep =  1
k =  9  rep =  2
k =  10  rep =  1
k =  10  rep =  2
k =  11  rep =  1
k =  11  rep =  2
k =  12  rep =  1
k =  12  rep =  2
k =  13  rep =  1
k =  13  rep =  2
k =  14  rep =  1
k =  14  rep =  2
k =  15  rep =  1
k =  15  rep =  2
k =  16  rep =  1
k =  16  rep =  2
k =  17  rep =  1
k =  17  rep =  2
k =  18  rep =  1
k =  18  rep =  2
k =  19  rep =  1
k =  19  rep =  2
k =  20  rep =  1
k =  20  rep =  2
k =  21  rep =  1
k =  21  rep =  2
k =  22  rep =  1
k =  22  rep =  2
k =  23  rep =  1
k =  23  rep =  2
k =  24  rep =  1
k =  24  rep =  2
k =  25  rep =  1
k =  25  rep =  2
k =  26  rep =  1
k =  26  rep =  2
k =  27  rep =  1
k =  27  rep =  2
k =  28  rep =  1
k =  28  rep =  2
k =  29  rep =  1
k =  29  rep =  2
k =  30 